In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
df = pd.read_csv('./data/processed/df.csv')
df

,id,title,review,score,sentiment
0,17998769,반지의 제왕: 왕의 귀환,판타지를 넘어 영화 역사에 남을 명작이다. 내가 이걸 왜 극장에서 못봤을까.... ...,10.0,NaN
1,17998767,나일 강의 죽음,이집트여행하는느낌의 영화,8.0,NaN
2,17998766,킹메이커,목포 연설 장면은 넋을 잃고 보게 된다.,10.0,NaN
3,17998765,기억의 밤,그저 그렇네요. 뻔한 반전과 결말.,4.0,NaN
4,17998764,더 배트맨,더 배트맨 조커처럼 몰입감이 있으면 추천 반대로 지루하면 비추천,1.0,NaN
...,...,...,...,...,...
9352,17988421,오! 수정,이해할 수 없는 남자와 이해해주는 여자,7.0,NaN
9353,17988420,이상한 나라의 수학자,배우들 연기력이 장난아님. 역시 최민식 배우는 클라스가 다름!,10.0,NaN
9354,17988419,블랙라이트,그래도 최근에 나왔던 영화들에 비해 내용이 되게 흥미진진했음 리암 니슨이 덜뛰는 대...,10.0,NaN
9355,17988418,더 배트맨,누구나 시작은 어설픈 법. 그래서 더 매력적인 히어로,10.0,NaN


In [9]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [10]:
okt = Okt()
okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

['오다', '이렇다', '것', '도', '영화', '라고', '차라리', '뮤직비디오', '를', '만들다', '게', '나다', '뻔']

In [ ]:
X_train = []
for sentence in tqdm(df['review']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)